# Imports

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container{max-width:91%!important;width:auto!important;}</style>"))

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Thesis')
folder = os.path.join('/content/drive/My Drive/Thesis')
!ls

Mounted at /content/drive
2-STREAM  Auto-encoder	code  Fast_load  TensorFlow


In [ ]:
import numpy as np
import torch as tc  # not an accepted convention
import os 
import math
import cv2
from tqdm import tqdm,trange
import json
from pathlib import Path
from PIL import Image
from torchvision.utils import make_grid
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import mimetypes
import torch.nn as nn
import torch
import random
import pandas as pd
import pickle
mimetypes.init()

In [ ]:
def show_imgs(ims, rows=1, figsize=(16,8), title=[None]):
    title = title*len(ims) if len(title) == 1 else title
    _,ax = plt.subplots(rows, len(ims)//rows, figsize=figsize)
    [show_img(im,ax_,title=tit) for im,ax_,tit in zip(ims,ax.flatten(),title)]
    return ax

In [ ]:
def show_img(im, ax=None, figsize=(8,8), title=None):
    if not ax: _,ax = plt.subplots(1,1,figsize=figsize)
    if len(im.shape)==2: im = np.tile(im[:,:,None], 3) 
    ax.imshow(im);
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    if title: ax.set_title(title)
    return ax

In [ ]:
def denorm(x, μ, σ):
    return (x * σ )+ μ

In [ ]:
def seed_everything(seed=2021):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

# Preprocessing

The function below takes a video file and extracts and saves all frames in the designated directory.

In [ ]:
def Imgs_Extractor(video_file, path='./', ext='.mp4', frames_dir='train_dir',frames_ext='.jpg'):
    os.makedirs(frames_dir, exist_ok=True) #creating directory for training or testing data 
    if extension not in video_file:
        video_file += extension #checking the of video file extension 
    cap = cv2.VideoCapture(path+video_file)   
    frame_Rate = cap.get(5) #frame rate
    os.makedirs(frames_dir+'/'+video_file, exist_ok=True) #creating directory per video
    counter = 0
    while(cap.isOpened()):
        frame_ID = cap.get(1) #current frame number
        ret, frame = cap.read() #reading frame and returning flag
        if (ret != True):
            break
        #Storing frames in frames_dir
        filename = frames_dir + '/' + video_file + '/' + video_file + '_frame{0}'.format(count)+frames_ext #frame label
        cv2.imwrite(filename,frame) #saving frame
        counter += 1
    cap.release()

Function that extracts paths corresponding to images only.

In [ ]:
def get_extensions_for_type(general_type):
    for ext in mimetypes.types_map:
        if mimetypes.types_map[ext].split('/')[0] == general_type:
            yield ext

In [ ]:
#Contains all possible extensions of images
IMAGE = tuple(get_extensions_for_type('image'))

Function that gets all file names per specified directory path.

In [ ]:
def get_files(path,Img_ext=IMAGE,flag_all=False):
     path = Path(path)
     if flag_all == False: 
        return [x for x in path.iterdir() if '.'+x.parts[-1].split('.')[-1] in Img_ext] #getting only paths that correspond to images
     else:
        return [x for x in path.iterdir()] #getting all paths
     

Preprocessing stage

In [ ]:
def all_paths(path,Img_ext):
    all_paths = [] #will contain all frames path of selected dataset
    all_vids = get_files(path,Img_ext,True) #gets all video paths of the dataset
    for vid in all_vids:
        all_frames = get_files(vid,Img_ext,False) #gets all frames paths of the dataset
        for frame in all_frames:
           all_paths.append(frame)
    return all_paths

In [ ]:
def Preprocessing(path,Img_ext,shape=(224,224)):
    paths=all_paths(path,Img_ext)
    #Image stacking
    Img_list = []
    for filename in tqdm(paths):
        img = Image.open(filename)
        img = img.resize(shape)
        img = np.array(img)
        Img_list.append(img)
    return Img_list

In [ ]:
p = os.getcwd()
p

'/content/drive/My Drive/Thesis'

In [ ]:
p = os.getcwd()
path = os.path.join(p,'code/Datasets/UCSDped1/UCSDped1/Test')
path

'/content/drive/My Drive/Thesis/code/Datasets/UCSDped1/UCSDped1/Test'

In [ ]:
Img_list = Preprocessing(path,IMAGE,(227,227))

100%|██████████| 7200/7200 [31:29<00:00,  3.81it/s]


In [ ]:
#p = os.getcwd()
#path = os.path.join(p,'testing_set_227x227.pkl')
#with open(path,'wb') as f: pickle.dump(Img_list, f)

In [ ]:
#Read training Data
p = os.getcwd()
path = os.path.join(p,'Fast_load/UCSDped1_train')
Img_list_training = pd.read_pickle (path)

In [ ]:
#Read Testing Data
#p = os.getcwd()
#path = os.path.join(p,'Fast_load/UCSDped1_test.pkl')
#Img_list_testing = pd.read_pickle(path)

In [ ]:
#Calculating mean μ and standard deviation σ.
numpy_Img_list_training = np.array(Img_list_training,dtype=np.float32)/255
μ = numpy_Img_list_training.mean(); σ = numpy_Img_list_training.std()
print(f'Data mean before normalization = {μ}, Data Std before normalization = {σ}')
#normlized_dataset = (numpy_Img_list-numpy_Img_list.mean())/numpy_Img_list.std()
#print(f'Data mean after normalization = {normlized_dataset.mean()}, Data Std after normalization = {normlized_dataset.std()}')

Data mean before normalization = 0.3750261962413788, Data Std before normalization = 0.2031460851430893


In [ ]:
#numpy_Img_list_testing = np.array(Img_list_testing,dtype=np.float32)/255

In [ ]:
#Mean and Std of training dataset:
μ,σ = 0.375,0.203

In [ ]:
print(f'number of frames for training dataset = {len(numpy_Img_list)}')

number of frames for training dataset = 6800


In [ ]:
def Normalization(Img_list,μ,σ):
    #Img_list=np.array(Img_list,dtype=np.float32)
    batch,height,width = Img_list.shape
    #Reshape to (224,224,batch_size)
    Img_list.resize(height,width,batch)
    #Normalize
    #Img_list=(Img_list-μ)/(σ)
    #Clip negative Values
    Img_list=np.clip(Img_list,0,1)
    #Final Reshaping
    Img_list.resize(batch,height,width)
    data = np.zeros((int(batch/10),10,height,width),dtype=np.float32)
    i, index = 0, 0
    for t in trange(batch):
        data[index,i,:,:] = (Img_list[t,...]-μ)/(σ)#/255
        i+=1
        if i%10 == 0 and i!=0:
            i=0;index+=1
    return data

In [ ]:
tr_ds= Normalization(numpy_Img_list_training,μ,σ)

100%|██████████| 6800/6800 [00:00<00:00, 14591.79it/s]


In [ ]:
#val_ds= Normalization(numpy_Img_list_testing,μ,σ)

100%|██████████| 7200/7200 [00:00<00:00, 13091.68it/s]


In [ ]:
tr_ds.shape

(680, 10, 224, 224)

In [ ]:
#shape of dataset
tr_ds = np.expand_dims(tr_ds, axis=1)
tr_ds.shape

(680, 1, 10, 224, 224)

In [ ]:
tr_ds.shape

(680, 1, 10, 224, 224)

In [ ]:
#val_ds = np.expand_dims(val_ds, axis=1)
#val_ds.shape

(720, 1, 10, 224, 224)

In [ ]:
#val_ds.shape

torch.Size([720, 1, 10, 224, 224])

In [ ]:
print(f'mean = {μ} ,Std = {σ}')

mean = 0.37718164920806885 ,Std = 0.20455783605575562


Training and Testing datasets

In [ ]:
tr_ds = tc.from_numpy(tr_ds)#;val_ds = tc.from_numpy(val_ds) #Transforming from numpy to tensor
tr_ds = tr_ds.type(tc.FloatTensor)#;val_ds = val_ds.type(tc.FloatTensor) #Casting tensors to Float32
tr_ds.dtype#,val_ds.dtype

torch.float32

In [ ]:
#Size of tensors: a.element_size() * a.nelement().
tr_ds.element_size()*8*224*224*10

16056320

# Auto-Encoder Model

In [ ]:
class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.

        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))



In [ ]:
class ConvLSTM(nn.Module):

    """

    Parameters:
        input_dim: Number of channels in input
        hidden_dim: Number of hidden channels
        kernel_size: Size of kernel in convolutions
        num_layers: Number of LSTM layers stacked on each other
        batch_first: Whether or not dimension 0 is the batch or not
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers
        Note: Will do same padding.

    Input:
        A tensor of size B, T, C, H, W or T, B, C, H, W
    Output:
        A tuple of two lists of length num_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    Example:
        >> x = torch.rand((32, 10, 64, 128, 128))
        >> convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
        >> _, last_states = convlstm(x)
        >> h = last_states[0][0]  # 0 for layer index, 0 for h index
    """

    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """

        Parameters
        ----------
        input_tensor: todo
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful

        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        b, _, _, h, w = input_tensor.size()

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            # Since the init is done in forward. Can send image size here
            hidden_state = self._init_hidden(batch_size=b,
                                             image_size=(h, w))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])
                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        return layer_output_list, last_state_list #------------------------------layer_output_list[0]

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

In [ ]:
class AutoEncoder(nn.Module): 
    def __init__(self):
        super().__init__()
        #Spatial Encoder
        self.spatial_encoder = nn.Sequential(
             nn.Conv3d(in_channels=1,  out_channels=128, kernel_size=(1,11,11), stride=(1,4,4), padding=(0,4,4),dilation=(1,1,1)),
             nn.BatchNorm3d(128),
             nn.Tanh(),
             nn.Conv3d(in_channels=128,  out_channels=256, kernel_size=(1,7,7), stride=(1,2,2), padding=(0,3,3)),
             nn.BatchNorm3d(256),
             nn.Tanh(),
             nn.Conv3d(in_channels=256,  out_channels=64, kernel_size=(1,5,5), stride=(1,2,2), padding=(0,2,2)),
             nn.BatchNorm3d(64),
             nn.Tanh()
            
            )
        #Convolutional_LSTM
        self.convlstm_stacked = ConvLSTM(input_dim=64,
                 hidden_dim=[64, 32, 64],
                 kernel_size=(3, 3),
                 num_layers=3,
                 batch_first=True,
                 bias=True,
                 return_all_layers=False)
        #Spatial Decoder
        self.spatial_decoder = nn.Sequential(
             nn.ConvTranspose3d(in_channels=64,  out_channels=256, kernel_size=(1,5,5), stride=(1,2,2), padding=(0,2,2),output_padding=(0,1,1)),
             nn.BatchNorm3d(256),
             nn.Tanh(),
             nn.ConvTranspose3d(in_channels=256,  out_channels=128, kernel_size=(1,7,7), stride=(1,2,2), padding=(0,3,3),output_padding=(0,1,1)),
             nn.BatchNorm3d(128),
             nn.Tanh(),
             nn.ConvTranspose3d(in_channels=128,  out_channels=1, kernel_size=(1,11,11), stride=(1,4,4), padding=(0,4,4),output_padding=(0,1,1)),
             #nn.BatchNorm3d(1),
             nn.Tanh(), 
            
        )
            
    def forward(self, x): 
        # here we define forward-pass logic
        #input of shape = (b, c, d , h, w)
        x = self.spatial_encoder(x)
        #(b, c, d , h, w) ---> (b, d, c, h, w)
        x = x.permute(0, 2, 1, 3, 4)
        x, last_states = self.convlstm_stacked(x)
        #(b, d, c, h, w) ---> (b, c, d , h, w)
        X = x[0]
        X = X.permute(0, 2, 1, 3, 4)
        X = self.spatial_decoder(X)
        return X

Visualize

# Training/Testing

one-epoch function that either trains or validates depending on input.

In [ ]:
# cell no 5
def one_epoch(net, loss, dl, opt=None, metric=None):
    
    if opt:
        net.train()  # only affects some layers
    else:
        net.eval()
        rq_stored = []
        for p in net.parameters():
            rq_stored.append(p.requires_grad)
            p.requires_grad = False
    
    L = []
    dl_it = iter(dl)
    for xb in tqdm(dl_it, leave=False):
        xb = xb.to(device)
        x_ = net(xb)
        l = loss(x_, xb)
        if opt:
            opt.zero_grad()
            l.backward()
            opt.step()
        L.append(l.detach().cpu().numpy())
        
        
    if not opt:
        for p,rq in zip(net.parameters(), rq_stored): p.requires_grad = rq
            
    return L    

Model

In [ ]:
device = tc.device('cuda' if tc.cuda.is_available() else 'cpu')
#model = AutoEncoder().to(device)

Hyper-parameters

In [ ]:
lr=1e-3 
wd=1e-3 
epoch=10 
loss=nn.L1Loss() 
batch_size = 8 

Data Loaders

In [ ]:
tr_dl  = DataLoader(tr_ds,  batch_size=batch_size, shuffle=True,  num_workers=2)
#val_dl  = DataLoader(val_ds,  batch_size=batch_size, shuffle=False,  num_workers=2)

Fit function

In [ ]:
# cell no 7
def fit(net, tr_dl, loss=nn.CrossEntropyLoss(), epochs=3, lr=3e-3, wd=1e-3, plot=True):   
    
    #opt = optim.Adam(net.parameters(), lr=lr, weight_decay=wd)
    opt = tc.optim.SGD( model.parameters(),lr= lr,momentum=0.9,weight_decay=wd) #SGD OPtimizer
    
    Ltr_hist, Lval_hist = [], []
    for epoch in trange(epochs):
        Ltr = one_epoch(net, loss, tr_dl,  opt)
        #Lval = one_epoch(net, loss, val_dl, None)
        Ltr_hist.append(np.mean(Ltr))
        #Lval_hist.append(np.mean(Lval))
        #print(f'epoch: {epoch}\ttraining loss: {np.mean(Ltr):0.4f}\tvalidation loss: {np.mean(Lval):0.4f}')
        print(f'epoch: {epoch+1}\ttraining loss: {np.mean(Ltr):0.4f}')
        
    # plot the losses     
    if plot:
        _,ax = plt.subplots(1,1,figsize=(16,4))
        ax.plot(1+np.arange(len(Ltr_hist)),Ltr_hist)
        #ax.plot(1+np.arange(len(Lval_hist)),Lval_hist)
        ax.grid('on')
        ax.set_xlim(left=1, right=len(Ltr_hist))
        ax.legend(['training loss']);
        #ax.legend(['training loss', 'validation loss']);
        
    return Ltr_hist#,Lval_hist

In [ ]:
Loss_tr = fit(model, tr_dl, loss, epoch, lr, wd, True)

Save Loss values of training

In [ ]:
p = os.getcwd()
path = os.path.join(p,'LOSS-MAE_SGD_80E_lr1_bs8_BN.pkl')
with open(path,'wb') as f: pickle.dump(Loss_tr, f)

Saving Model

In [ ]:
#Saving
checkpoint = {
"epoch": 80,
"model_state": model.state_dict(),
#"optim_state": optim.state_dict()
}
#print(optim.state_dict())
FILE = "SGD_checkpoint_Epoch80.pth"
torch.save(checkpoint, FILE)

Loading Model

In [ ]:
#Loading
p = os.getcwd()
FILE = os.path.join(p,'2-STREAM/SGD_checkpoint_Epoch50.pth')
model = AutoEncoder()
checkpoint = torch.load(FILE)
model.load_state_dict(checkpoint['model_state'])
model.to(device)
epoch = checkpoint['epoch']
print(f'checkpoint epoch = {epoch}')
#model.eval()
# - or -
# model.train()

checkpoint epoch = 50


In [ ]:
Loss_tr = fit(model, tr_dl, loss, epoch, lr, wd, True)

Plot Regularity score.

In [ ]:

def Regularity_score(net,dl):
  """
  Regularity Score = 1 - (e_t - min@t(e_t))/max@t(e_t)
  """
  e_t = []
  dl_it = iter(dl)
  net.eval()
  for X in tqdm(dl_it,leave=False):
     output = net(X)
     X = X.numpy().squeeze()
     output = output.detach().numpy().squeeze()
     bs,depth,height,width = X.shape[0],X.shape[-3],X.shape[-2],X.shape[-1]
     for B in trange(bs):
       for i in trange(depth):
         #TODO
         a = X[B,i,:,:].reshape(height*width,1)
         b = output[B,i,:,:].reshape(height*width,1)
         e_xyt = np.linalg.norm(a-b,axis=0,ord=2)
         e_t.append(e_xyt[0])

  e_t_min = min(e_t)
  e_t_max = max(e_t)
  reg_scores = []
  for i in trange(len(e_t)):
     reg_scores.append(1 - ((e_t[i]-e_t_min)/e_t_max))
  
  return reg_scores 

In [ ]:
#reg_scores=Regularity_score(model,dl)

Plotting the Regularity Score

In [ ]:
def plot(reg_scores):
  _,ax = plt.subplots(1,1,figsize=(16,4))
  ax.plot(1+np.arange(len(reg_scores)),reg_scores,color = 'red')
  ax.grid('on')
  ax.set_xlabel("frame number")
  ax.set_ylabel("regularity score")
  ax.set_title( "Regularity Score per frame")

#Save and Load Checkpoint

In [ ]:
#Saving
checkpoint = {
"epoch": 30,
"model_state": model.state_dict(),
#"optim_state": optim.state_dict()
}
#print(optim.state_dict())
FILE = "ch_checkpoint_Epoch30.pth"
torch.save(checkpoint, FILE)

In [ ]:
#Loading
p = os.getcwd()
FILE = os.path.join(p,'Fast_load/checkpoint_Epoch20.pth')
model = AutoEncoder()
#optim = torch.optim.SGD(model.parameters(), lr= lr,momentum=0.9,weight_decay=wd)
checkpoint = torch.load(FILE)
model.load_state_dict(checkpoint['model_state'])
model.to(device)
#optimizer.load_state_dict(checkpoint['optim_state'])
#epoch = checkpoint['epoch']
#model.eval()
# - or -
# model.train()

In [ ]:
#p = os.getcwd()
#path = os.path.join(p,'Fast_load/loss_sgd_10E_lr1_bs4_BN.pth')
#Img_list = pd.read_pickle(path)

# Trials

In [ ]:
path = '/content/drive/My Drive/Thesis/code/005.tif'
img = Image.open(path)

In [ ]:
p = os.getcwd()
path = os.path.join(p,'code/005.tif')
path
img = Image.open(path)
img = img.resize((227,227))
img.size

(227, 227)

In [ ]:
#-----------------------------TRIAL-------------------------------------------
class trial(nn.Module): 
    def __init__(self):
        super().__init__()
        self.spatial_encoder = nn.Sequential(
             nn.Conv3d(in_channels=1,  out_channels=128, kernel_size=(1,11,11), stride=(1,4,4), padding=(0,2,2)),
             nn.BatchNorm3d(128),
             nn.Tanh(),
            )
        self.temporal_encoder = nn.Sequential()
        self.spatial_decoder = nn.Sequential(
             nn.Conv3d(in_channels=64,  out_channels=256, kernel_size=(1,5,5), stride=(1,2,2), padding=1),
             nn.BatchNorm3d(256),
             nn.Tanh(),
             nn.Conv3d(in_channels=256,  out_channels=128, kernel_size=(1,7,7), stride=(1,2,2), padding=2),
             nn.BatchNorm3d(128),
             nn.Tanh(),
             nn.Conv3d(in_channels=128,  out_channels=1, kernel_size=(1,11,11,), stride=(1,4,4), padding=2),
             nn.BatchNorm3d(1),
             nn.Tanh(),
            
        )
            
    def forward(self, x): 
        # here we define forward-pass logic
        x = self.spatial_encoder(x)
        #x, last_states = self.convlstm_1(x)
        #x, last_states = self.convlstm_2(x)
        #x, last_states = self.convlstm_3(x)
        #x = self.spatial_decoder(x)
        return x
#--------------------------------------------------------------TRIAL------------------------------------------------------------------

In [ ]:
spatial_decoder = nn.Sequential(
             nn.ConvTranspose3d(in_channels=64,  out_channels=256, kernel_size=(1,5,5), stride=(1,2,2), padding=(0,2,2),output_padding=(0,1,1)),
             nn.BatchNorm3d(256),
             nn.Tanh(),
             nn.ConvTranspose3d(in_channels=256,  out_channels=128, kernel_size=(1,7,7), stride=(1,2,2), padding=(0,3,3),output_padding=(0,1,1)),
             nn.BatchNorm3d(128),
             nn.Tanh(),
             nn.ConvTranspose3d(in_channels=128,  out_channels=1, kernel_size=(1,11,11), stride=(1,4,4), padding=(0,4,4),output_padding=(0,1,1)),
             #nn.BatchNorm3d(1),
             nn.Tanh()
        )

In [ ]:
#(b, c, d , h, w)
#x_decode = torch.rand((1, 64, 10, 14, 14))
x_decode = torch.rand((1, 64, 10, 14, 14))
outy = spatial_decoder(x_decode)
outy.size()

torch.Size([1, 1, 10, 224, 224])

In [ ]:
spatial_encoder = nn.Sequential(
             nn.Conv3d(in_channels=1,  out_channels=128, kernel_size=(1,11,11), stride=(1,4,4), padding=(0,4,4),dilation=(1,1,1)),
             nn.BatchNorm3d(128),
             nn.Tanh(),
             nn.Conv3d(in_channels=128,  out_channels=256, kernel_size=(1,7,7), stride=(1,2,2), padding=(0,2,2),dilation=(1,1,1)),
             nn.BatchNorm3d(256),
             nn.Tanh(),
             nn.Conv3d(in_channels=256,  out_channels=64, kernel_size=(1,5,5), stride=(1,2,2), padding=(0,2,2)),
             nn.BatchNorm3d(64),
             nn.Tanh()
            )

In [ ]:
#(b, c, d , h, w)
x_encode = torch.rand((1, 1, 10, 224, 224))
outy = spatial_encoder(x_encode)
outy.size()

In [ ]:
#(b, c, d , h, w) ---> (b, d, c, h, w)
x_encode = torch.rand((1, 64, 10, 224, 224))
x_encode = x_encode.permute(0, 2, 1, 3, 4)

In [ ]:
model = ConvLSTM(input_dim=64,
                 hidden_dim=[64, 32, 64],
                 kernel_size=(3, 3),
                 num_layers=3,
                 batch_first=True,
                 bias=True,
                 return_all_layers=False)

In [ ]:
h_next,states = model(x_encode)

In [ ]:
len(states[0]),len(h_next)

(2, 1)

In [ ]:
class AutoEncoder(nn.Module): 
    def __init__(self):
        super().__init__()
        #Spatial Encoder
        self.spatial_encoder = nn.Sequential(
             nn.Conv3d(in_channels=1,  out_channels=128, kernel_size=(1,11,11), stride=(1,4,4), padding=(0,2,2)),
             nn.BatchNorm3d(128),
             nn.Tanh(),
             nn.Conv3d(in_channels=128,  out_channels=256, kernel_size=(1,7,7), stride=(1,2,2), padding=(0,1,1)),
             nn.BatchNorm3d(256),
             nn.Tanh(),
             nn.Conv3d(in_channels=256,  out_channels=64, kernel_size=(1,5,5), stride=(1,2,2), padding=(0,1,1)),
             nn.BatchNorm3d(64),
             nn.Tanh()
            )
        #LSTM
        self.convlstm_1 = ConvLSTM(64, 32, (3,3), 1, True, True, False)
        self.convlstm_2 = ConvLSTM(32, 32, (3,3), 1, True, True, False)
        self.convlstm_3 = ConvLSTM(32, 64, (3,3), 1, True, True, False)
        #Spatial Decoder
        self.spatial_decoder = nn.Sequential(
             nn.ConvTranspose3d(in_channels=64,  out_channels=256, kernel_size=(1,5,5), stride=(1,2,2), padding=(0,1,1)),
             nn.BatchNorm3d(256),
             nn.Tanh(),
             nn.ConvTranspose3d(in_channels=256,  out_channels=128, kernel_size=(1,7,7), stride=(1,2,2), padding=(0,2,2)),
             nn.BatchNorm3d(128),
             nn.Tanh(),
             nn.ConvTranspose3d(in_channels=128,  out_channels=1, kernel_size=(1,11,11), stride=(1,4,4), padding=(0,2,2)),
             #nn.BatchNorm3d(1),
             nn.Tanh(),
            
        )
            
    def forward(self, x): 
        # here we define forward-pass logic
        x = self.spatial_encoder(x)
        #(b, c, d , h, w) ---> (b, d, c, h, w)
        x = x.permute(0, 2, 1, 3, 4)
        x, last_states = self.convlstm_1(x)
        x, last_states = self.convlstm_2(x)
        x, last_states = self.convlstm_3(x)
        #(b, d, c, h, w) ---> (b, c, d , h, w)
        x = x.permute(0, 2, 1, 3, 4)
        x = self.spatial_decoder(x)
        return x

In [ ]:
X = np.random.rand(1,1,5,3)
X = np.round(X,3)

In [ ]:
p = os.getcwd()
path = os.path.join(p,'code/loly')
path

'/content/drive/My Drive/Thesis/code/loly'

In [ ]:
with open(path,'wb') as f: pickle.dump(X, f)

In [ ]:
test = pd.read_pickle (path)
test.shape

(1, 1, 5, 3)

In [ ]:
# cell no 5
def one_epoch(net, loss, dl, opt=None, metric=None):
    
    if opt:
        net.train()  # only affects some layers
    else:
        net.eval()
        rq_stored = []
        for p in net.parameters():
            rq_stored.append(p.requires_grad)
            p.requires_grad = False
    
    L, M = [], []
    dl_it = iter(dl)
    for xb, yb in tqdm(dl_it, leave=False):
        xb, yb = xb.cuda(), yb.cuda()
        y_ = net(xb)
        l = loss(y_, yb)
        if opt:
            opt.zero_grad()
            l.backward()
            opt.step()
        L.append(l.detach().cpu().numpy())
        if metric: M.append(metric(y_, yb).cpu().numpy())
        
    if not opt:
        for p,rq in zip(net.parameters(), rq_stored): p.requires_grad = rq
            
    return L, M

# cell no 7
def fit(net, tr_dl, val_dl, loss=nn.CrossEntropyLoss(), epochs=3, lr=3e-3, wd=1e-3, plot=True):   
    
    opt = optim.Adam(net.parameters(), lr=lr, weight_decay=wd)
    
    Ltr_hist, Lval_hist = [], []
    for epoch in trange(epochs):
        Ltr,  _    = one_epoch(net, loss, tr_dl,  opt)
        Lval, Aval = one_epoch(net, loss, val_dl, None, accuracy)
        Ltr_hist.append(np.mean(Ltr))
        Lval_hist.append(np.mean(Lval))
        print(f'epoch: {epoch}\ttraining loss: {np.mean(Ltr):0.4f}\tvalidation loss: {np.mean(Lval):0.4f}\tvalidation accuracy: {np.mean(Aval):0.2f}')
        
    # plot the losses     
    if plot:
        _,ax = plt.subplots(1,1,figsize=(16,4))
        ax.plot(1+np.arange(len(Ltr_hist)),Ltr_hist)
        ax.plot(1+np.arange(len(Lval_hist)),Lval_hist)
        ax.grid('on')
        ax.set_xlim(left=1, right=len(Ltr_hist))
        ax.legend(['training loss', 'validation loss']);
        
    return Ltr_hist, Lval_hist    


In [ ]:
#accuracy = lambda y_,yb: (y_.max(dim=1)[1] == yb).float().mean()